# Straka Bubble Test 🫧🫧

<div class="alert alert-block alert-success">

## Part 2: Generating the Initialization `.yaml` File

There are two main components needed to run a simulation with $\texttt{PADDLE}$. The first is a an initialization file (in the form of a `.yaml` file), and the second is the execution python script (`.py` or `.ipynb` file) needed to run the simulation. 

In this part of the tutorial, we will be going step-by-step through the contents of the `.yaml` file configuration for the Straka bubble test. Many of the elements and concepts we find in the `.yaml` file for Straka will continue to be in future tutorial notebooks as well. 

<div class="alert alert-warning">

## $\texttt{PADDLE}$ Straka Initialization (Generation of .yaml file)

A `.yaml` file is, in essence, a dictionary of set options for the simulation. 

In this notebook, we will be generating the dictionary for the Straka bubble test from scratch and convert it into a `.yaml` file at the end.

However, as we will see in **Part 5** of the Straka bubble test tutorial (where we run the Straka bubble test using terminal commands instead of a Jupyter notebook), the `.yaml` can be edited directly instead of being built in a Jupyter notebook. 

We suggest having the `straka.yaml` file (located in this folder) pulled up while following the tutorial below. 

</div>

<div class="alert alert-info">

### Define Simulation Domain Geometry ($\texttt{geometry}$)

The first dictionary entry in the `.yaml` file, $\texttt{geometry}$, defines the geometry of the simulation domain. 

The first thing we define is the domain (size and resolution) of the simulation. In this case, its the spatial dimension and resolution of the background atmosphere.  

For our background atmosphere, lets define a 2D atmosphere that is 25.6 kilometers (15 miles) wide and 6.4 km (4 miles) high. 
In $\texttt{PADDLE}$, the default unit system is SI. 

</div>

In [1]:
# Here we define spatial dimensions 
# Each has a min and max value, defining the cartesian box

# Initialize a cartesian geometry (a box, or a cube)
geometry_type = 'cartesian'

# Spatial Dimensions ('bounds')

# x1 = Height (often denoted with 'z')
x1min = 0
x1max = 6.4e3 # in meters

# x2 = Width (often denoted with 'x')
x2min = 0
x2max = 25.6e3 # in meters

# x3 = Depth (often denoted with 'y')
# While our atmosphere is 2D, we still need to define the depth dimension. 
# Be defining our resolutions, we will ensure the geometry is 2D instead of 3D
# The depth value here is arbitrary
x3min = 0
x3max = 25.6e3

# Spatial Resolutions ('cells')
# Here, we will define the identical vertical and horizontal resolutions

# nx1 = Number of Cells in Height Dimension
nx1 = 64 # 1 cell per 0.1 km 

# nx2 = Number of Cells in Width Dimension 
nx2 = 256 # 1 cell per 0.1 km 

# nx3 = Number of Cells in Depth Dimension 
# Here we want a 2D simulation, so we set this to 1 
nx3 =1 # 1 to reduce simulation to 2D

# nghost = Number of ghost zones
# The ghost zones are additional cells that pad the interior simulation domain 
# I.e. if a simulation is 40x40 it would be 46x46 due to 3 additional cells on each side 
# A good default number here is 3
nghost = 3

Now we actually create the dictionary entry for geometry from the variables above, so we can generate the `.yaml` file from a dictionary at the end of this section.

In [2]:
geometry_dict = {'type': geometry_type,
                 'bounds': {'x1min': x1min, 'x1max': x1max,
                            'x2min': x2min, 'x2max': x2max, 
                            'x3min': x3min, 'x3max': x3max},
                 'cells':  {'nx1': nx1, 'nx2': nx2, 'nx3': nx3, 'nghost': nghost}}

<div class="alert alert-info">

### Define Dynamics ($\texttt{dynamics}$)

The second dictionary entry, $\texttt{dynamics}$, defines the dynamical properties of the simulation. 

In this context, $\texttt{dynamics}$ means properties like the equation of state of the atmosphere, as well as defining the algorithms that are utilized to solve the hydrodynamics of the simulation. 

The $\texttt{dynamics}$ dictionary entry is composed of multiple sub-dictionaries, defined below.

</div>

#### Dynamics sub-dictionary: Equation-of-State Properties 

For our Straka bubble case, we will assume the entire atmosphere is an ideal gas. As discussed in **Part 1**, ideal gases follow the ideal gas law, and all of their state variables (pressure, temperature, volume, internal energy, etc) can be defined entirely by two variables: 

1. The adiabatic index $\gamma$
2. Molecular weight $M$

We will define the parameters above as we did in **Part 1**.

In future tutorials, we will explore other equation-of-states included in $\texttt{PADDLE}$. 

In [3]:
# Equation of State (EOS) Properties

# We first must set up the equation-of-state of the simulation gas. 
# The equation of state is a thermodynamic equation that relates state variables 
# such as pressure, temperature, volume, and internal energy. 

# Here we use the ideal gas law equation-of-state, which reads as:
# P = rho R T
# where P is pressure; rho is density; R is the specific gas constant; T is temperature
# We will discuss other equations of state in other tutorials
eos_type = 'ideal-gas'

# For ideal gases, you only need two variables, the adiabatic index and molecular weight 

# gammad is the adiabatic index
# It is equal to c_p/c_v = C_p/C_v (heat capacity const p/heat capacity const v)
# (note that lowercase c = specific heat capacity [depends on mass], uppercase C = heat capacity)
# The adiabatic index 1.66 for monoatomic gasses (He, Ar) and 1.4 for diatomic gasses (N, O, Air)
gammad = 1.4 

# molecular weight (kg/mol)
# This is the molecular weight of Earth's atmosphere 
weight = 29.3e-3 # in kg/mol

# Make Sub-Dictionary 
equation_of_state_dict = {'type': eos_type, 'gammad': gammad, 'weight':weight}

#### Dynamics sub-dictionaries: Vertical Projection, Reconstruct, and Riemann Solver Properties 

For this initial tutorial, the following sub-dictionaries of $\texttt{dynamics}$ (which determine different algorithms that are used in the simulation, and therefore numerical stability) can be taken as-is, though we will provide short introductions to the following: 

The $\texttt{vertical-projection}$ defines a numerical method that is used in non-hydrostatic codes. In short, this numerical method subtracts a hydrostatically balanced atmosphere and lets non-hydrostatic pressure be the force that drives air flow. The default option is to use temperature as the variable for calculating a hydrostatically balanced atmosphere.

The $\texttt{reconstruct}$ properties define the algorithm to compute hydrodynamic variables at cell boundaries in the simulation from properties like cell averages. Here we use the [WENO5](https://en.wikipedia.org/wiki/WENO_methods) (fifth-order accurate weighted essentially non-oscillation) algorithm, which is generally stable for a variety of applications.

Finally we have the $\texttt{riemann-solver}$, which computers fluxes given hydrodynamic variables at cell boundaries. There are two main options: HLLC (Harten-Lax-van Leer-Contact) and LMARS (Low Mach number Approximate Riemann Solver). HLLC is generally more stable but LMARS is faster.

A few extra configuration options, such a *pressure_margin*, *scale*, *shock* are fine-tune parameters for each algorithm.

In [4]:
# Vertical Projection Properties 

# Vertical projection type 
vertical_projection_type = 'temperature'

pressure_margin = 1e-6

# Make Sub-Dictionary 
vertical_projection_dict = {'type': vertical_projection_type,
                            'pressure-margin': pressure_margin
                            }


In [5]:
# Reconstruct Properties 

# Uses the weno5 algorithm 

# Vertical Reconstruction 
vertical_reconstruct_type = 'weno5'
vertical_reconstruct_scale = False
vertical_reconstruct_shock = False

# Horizontal Reconstruction 
horizontal_reconstruct_type = 'weno5'
horizontal_reconstruct_scale = False
horizontal_reconstruct_shock = False

# Make Sub-Dictionary 
reconstruct_dict = {'vertical'  : {'type' : vertical_reconstruct_type,
                                   'scale': vertical_reconstruct_scale,
                                   'shock': vertical_reconstruct_shock},
                    'horizontal': {'type' : horizontal_reconstruct_type,
                                   'scale': horizontal_reconstruct_scale,
                                   'shock': horizontal_reconstruct_shock}}

In [6]:
# Riemann Solver 

# LMARS solver (Low Mach number Approximate Riemann Solver)
# was developed to simulate low-speed flows that are more typical for atmospheric simulations
riemann_type = 'lmars'

# Make Sub-Dictionary 
riemann_solver_dict = {'type': riemann_type}

Now we create the dictionary object for the future .yaml file

In [7]:
# Combine the dictionaries above into a larger dynamics dictionary 

dynamics_dict = {'equation-of-state': equation_of_state_dict,
                 'vertical-projection': vertical_projection_dict,
                 'reconstruct': reconstruct_dict,
                 'riemann-solver': riemann_solver_dict}

<div class="alert alert-info">

### Define Boundary Conditions ($\texttt{boundary-condition}$)

The third dictionary entry, $\texttt{boundary-condition}$, defines how waves interact with the boundaries of the simulation domain. 

In particular, there are three main kinds of boundary conditions we can define for each wall of the simulation domain: 

1. Reflecting : This is a solid wall boundary that waves cannot cross. 

2. Periodic : This is a wrapping boundary. I.e., if the right and left walls of the box domain are periodic, waves that hit the right hand side of the domain will appear on the left hand side. If used for our 2D case, that means the simulation domain will be essentially a cylinder, instead of a box. 

3. Outflow: Waves leave the simulation domain forever

The main sub-dictionary we will define here is the $\texttt{external}$ boundary conditions. There also exists $\texttt{internal}$ boundary conditions, which defines how waves interact with boundaries inside the simulation domain (like topography), which we will explore in future tutorials simulating rocky planet atmospheres. 

</div>

In [8]:
# External Properties 

# x1 (height) inner and outer properties 
x1_inner = 'reflecting' # Bottom Wall
x1_outer = 'reflecting' # Top Wall

# x2 (width) inner and outer properties 
x2_inner = 'reflecting' # Left Wall
x2_outer = 'reflecting' # Right Wall

# x3 (depth) inner and outer properties 
x3_inner = 'reflecting' # Towards us Wall
x3_outer = 'reflecting' # Away from us Wall


Now we create the dictionary object for the future .yaml file

In [9]:
boundary_condition_dict = {'external': {'x1-inner': x1_inner,
                                         'x1-outer': x1_outer,
                                         'x2-inner': x2_inner,
                                         'x2-outer': x2_outer,
                                         'x3-inner': x3_inner,
                                         'x3-outer': x3_outer}}

<div class="alert alert-info">

### Define Integration Properties ($\texttt{integration}$)

The fourth dictionary entry, $\texttt{integration}$, determines how time-stepping is solved and handled in the simulation. In particular, it also sets how long a simulation is run for. 

For this tutorial, there is no need to change the integration-specific parameters below, though we will provide an explanation of each here: 

$\texttt{integration-type}$ is a strong-stability preserving third-order Runge-Kutta method ('rk3') (for mathematical details, see [link](https://gkeyll.readthedocs.io/en/latest/dev/ssp-rk.html)). This is the default integration type, and there are not too many reasons to change this. 

$\texttt{cfl}$ is the [Courant-Friedrich-Lewy](https://en.wikipedia.org/wiki/Courant%E2%80%93Friedrichs%E2%80%93Lewy_condition) number which determines the stability of the time-stepping scheme. It is the the ratio of how far a wave/particle can travel in a single time step vs the local cell size. You want this number to be less than 1, where a cfl larger than 1 means information travels more than one cell in a time step, which is not stable.

$\texttt{implicit-scheme}$ determines which Vertical-Implicit correction (VIC) scheme is used. While not important for the Straka case where the vertical and horizontal resolutions are the same, usually atmospheric simulations have a large aspect ratio between horizontal and vertical directions that causes the integration time step to be limited by vertical motions. The first implementation of VIC in $\texttt{SNAP}$ can be found in [Ge et al. (2020)](https://iopscience.iop.org/article/10.3847/1538-4357/ab9ec7/pdf). In future tutorials, we will explore different options here (0 = everything is explicit, 1 = simplified vertical implicit for cartesian cases, and 9 = full vertical implicit for cubed-sphere simulations).

</div>

In [10]:
# Integration properties 

# Integration type is a third-order Runge-Kutta method
integration_type = 'rk3'

# cfl = stability condition (Courant-Friedrich-Lewy number)
cfl = 0.9

# implicit_scheme 
# This is tied to the idea of Vertical-Implicit correction schemes
# 0 = everything explicit 
implicit_scheme = 0

Here we define properties that determine how long a simulation runs and how often diagnostic information is printed out. 

There are two important distinctions to be made: 

1. $\texttt{cycles}$: A cycle is an integration step, where the simulation time that has elapsed (*dt*) is determined in part by the cfl number above. 

2. $\texttt{time}$: Time here is the number of seconds that have elapsed in the simulation, not amount of time that has elapsed in real life. 

In [11]:
# nlim 
# Number of cycles to run model 
# -1 = run until time limit
nlim = -1 

# How long we want the simulation to run before it terminates
# Here, we set it at 900 seconds
tlim = 900

# ncycle_out 
# frequency of printing out diagnostic information to terminal 
# cycle = integration step
ncycle_out = 10

Now we create the dictionary object for the future .yaml file

In [12]:
integration_dict = {'type': integration_type,
                    'cfl': cfl,
                    'implicit-scheme': implicit_scheme,
                    'nlim': nlim,
                    'tlim': tlim,
                    'ncycle_out':10}

<div class="alert alert-info">

### Define Forcing ($\texttt{forcing}$ )

The fifth dictionary entry, $\texttt{forcing}$, defines what deviates the simulation from its initial equilibrium state. In general, all simulations will include a constant (with height) gravity. We will explore other types of forcings in future tutorials. 

</div>

In [13]:
# Forcing properties 
# Here, we only have the 'const-gravity' sub-dictionary 

# Gravity in the x1 (height) direction
# A negative sign means that gravity points downward
gravity_1 = -9.8 # in m/s^2

Now we create the dictionary object for the future `.yaml` file

In [14]:
forcing_dict = {'const-gravity': {'grav1': gravity_1}}

<div class="alert alert-info">

### Define Outputs ($\texttt{outputs}$)

The final dictionary entry, $\texttt{outputs}$, determines what sorts of files are produced by the simulation. This portion is extremely important to define carefully, as it determines how often outputs are produced and what sort of variables are saved for future analysis. 

When you run $\texttt{PADDLE}$ code, two sorts of files will be produced. 

1. Restart files: These files (denoted by **.restart**) will be produced every $\texttt{dt-restart}$ (in simulation time). These files can be used to restart a simulation from a specific 'checkpoint' and are essential when running larger simulations (where computers can time out). 

2. Netcdf progress files: These files are produced every $\texttt{dt-netcdf}$ (in simulation time). They store all the variables defined in $\texttt{variables-track}$. Once a simulation has finished, these files can be combined into a single .nc file and are deleted. Below, we define that we want to track the primitive variables ($\texttt{prim}$) which includes density, pressure, and velocities in the three directions (v1-v3). We also can define user output variables ($\texttt{uov}$) which are defined explicitly when we create our $\texttt{PADDLE}$ execution code in **Part 3**. 

</div>

In [15]:
# Output properties 

# Two different kind of output files. Restart files, and netcdf progress files

# How often we want .restart files to be generated
# Here, we define a restart file to be generated every 300 simulations seconds
# Resulting in a total of 5 restart files at 0, 300, 600, and 900 seconds 
# (and an additional final, and in this case redundant, one at t_lim = 900s)
dt_restart = 300

# Then we define how often we generate .nc files that track specific variables
# We first define which variables we want to be track 
# prim = primitive variables
# density, three velocities (v1-v3), pressure
# uov = user output variable (Athena++ nomenclature) 
# tied to call_user_output in py script
variables_track = ['prim','uov']

# Then we define how often we want the files to be generated 
# Here, its every 30 simulation seconds
# After the simulation is finished, 
# these output files are then compressed into a single nc file that 
# can be analyzed
dt_netcdf = 30

Now we create the dictionary object for the future .yaml file

In [16]:
outputs_dict = [{'type': 'restart', 'dt': dt_restart},
               {'type': 'netcdf', 'variables' : variables_track, 'dt': dt_netcdf}]

<div class="alert alert-info">

### Create the input `.yaml` file 

Here, we generate the `.yaml` file from the dictionaries we defined above. 

If you have the `straka.yaml` pulled up, you'll notice how we have been following the structure of that file in this notebook here. 

In this notebook, we explicitly generate the input `.yaml` file from dictionaries defined in this Jupyter notebook. However, in the future, we can just edit a `.yaml` file directly (additionally, the pre-made `.yaml` files should be more human readable than the one we generate here).

<div>

In [20]:
import yaml

full_dictionary = {'geometry': geometry_dict,
                   'dynamics': dynamics_dict,
                   'boundary-condition': boundary_condition_dict,
                   'integration': integration_dict,
                   'forcing': forcing_dict,
                   'outputs': outputs_dict,}

file_path = "straka_made_in_notebook.yaml"
with open(file_path, "w") as file_handler:
    yaml.dump(full_dictionary, file_handler, sort_keys=False)

<div class="alert alert-block alert-success">

Now we have our `.yaml` file with all the options needed to execute the Straka bubble simulation. 

On we go to **Part 3**, where we discuss how to set up the `.py` (or `.ipynb`) execution script that is used to run a  $\texttt{PADDLE}$ simulation.